### KNU 한국어 감성사전 요약

- **작성자 및 소속**: 온병원, 박상민, 나철원 / 군산대학교 소프트웨어융합공학과 Data Intelligence Lab
- **작성일**: 2018.05.11
- **홈페이지**: [Data Intelligence Lab](http://dilab.kunsan.ac.kr/)

#### 개요

- **목적**: 인간의 기본적인 감정 표현을 나타내는 긍정과 부정어로 구성. 특정 도메인(음식, 여행지 등)이 아닌 보편적인 감성 표현에 초점.
- **구성원리**: 국립국어원 표준국어대사전, 김은영의 연구, SentiWordNet, SenticNet-5.0, 온라인 축약어 및 이모티콘에서 긍부정어 추출.
- **콘텐츠**: 총 14,843개의 단어와 문구에 대한 감성 분류 및 강도 측정.

#### 특징

- **다양성**: 1-gram부터 n-gram, 축약어, 이모티콘까지 다양한 형태의 긍부정어 포함.
- **보편성**: 모든 도메인에 적용 가능한 보편적 긍부정어 사용.

#### 개발 방법

- **기술**: Bi-LSTM 딥 러닝 모델을 사용하여 표준국어대사전의 단어 뜻풀이를 긍정, 중립, 부정으로 분류.
- **평가**: 최소 3명의 평가자가 리커트 척도를 사용하여 감성을 판별, 이의가 있을 시 토론을 통해 합의.

#### 통계

- **단어 분류**:
  - 1-gram 긍부정어: 6,223
  - 2-gram 긍부정어: 7,861
  - 긍부정 어구, 문형, 축약어, 이모티콘 등 포함
- **감성 분포**:
  - 매우 긍정: 2,597
  - 긍정: 2,266
  - 중립: 154
  - 부정: 5,029
  - 매우 부정: 4,797

#### 참고문헌

- 표준국어대사전, 긍부정 축약어, 이모티콘, SentiWordNet, SenticNet, 감정단어사전, 김은영의 연구, 트위터 형태소 분석기 등.


In [6]:
! pip install konlpy

In [9]:
# 필요한 라이브러리를 불러옵니다.
import pandas as pd  # 데이터 처리를 위한 pandas 라이브러리
from konlpy.tag import Okt  # 한국어 자연어 처리를 위한 konlpy의 Okt 모듈
# Okt 객체를 생성합니다. 한국어 텍스트를 형태소 단위로 분석할 수 있게 해줍니다.
okt = Okt()
# 엑셀 파일로부터 데이터를 읽어와 DataFrame으로 저장합니다.
data = pd.read_excel('/content/빅카인즈_인공지능_뉴스.xlsx')
# 뉴스 제목과 본문을 합쳐서 새로운 컬럼 '제목_본문'을 생성합니다.
data['제목_본문'] = data['제목'] + data['본문']


In [10]:
data['제목_본문']

0       트럼프, 주황 죄수복 입고 바닥 청소 이 사진 정체 알고보니도널드 트럼프 전 미국 ...
1       트럼프 체포됐다고? AI로 만든 가짜 사진 퍼지며 논란도널드 트럼프 전 미국 대통령...
2       트럼프가 체포됐다? “다리가 이상해” AI 가짜 이미지에 ‘깜빡’‘성추문 입막음’ ...
3       ‘선허용 후규제’ AI법 논란  시민단체 “사회 경제 손실 초래”정보 인권관련 시민...
4       비브스튜디오스, 몬스터유니온과 드라마 공동제작 MOU 체결KBS 대하사극 ‘고려거란...
                              ...                        
5606    경기도정 ‘디지털 전환’ 청사진 제시앞으로 경기도에선 인공지능을 활용해 민원 서식을...
5607    AI 횡단보도가 휠체어 감지해 보행신호 연장울산시 남구 울산시청 앞 횡단보도. 보행...
5608    [삶의 향기] ‘쓰봉’과 ‘우와기’ 기억하시나요우연한 기회에 차로 두어 시간을 가야...
5609    [라이프 트렌드&] “신제품을 얻은 듯 만족”   쓸수록 진화하는 ‘UP가전’LG전...
5610    공무원 추첨제 꺼낸 약사 출신, '약사도 추첨제 하셈' 지적에 [박한슬의 댓글 읽어...
Name: 제목_본문, Length: 5611, dtype: object

In [11]:
# 데이터 전처리 함수입니다. 텍스트에서 형용사만을 추출하여 반환합니다.
def preprocess_data(text):
    morphs = okt.pos(text)  # 텍스트를 형태소 분석합니다.
    words = [word for word, tag in morphs if tag == 'Adjective' and len(word) > 1]  # 형용사만 추출합니다.
    return ' '.join(words)  # 추출된 형용사들을 공백으로 구분하여 하나의 문자열로 반환합니다.

# 모든 뉴스 데이터에 대해 위의 전처리 함수를 적용합니다.
data['preprocessing'] = data['제목_본문'].astype('str').apply(preprocess_data)



In [12]:
data['preprocessing']

0                    있다고
1                     있는
2         이상해 있다 있다고 강하게
3                 있는 미치는
4                    놀라운
              ...       
5606                 필요한
5607            있다고 신속하고
5608    어떤 같기도 어리둥절하던 같은
5609                    
5610                 친절한
Name: preprocessing, Length: 5611, dtype: object

In [15]:

# 감성 사전을 로딩합니다. 각 단어의 긍정, 부정, 중립 감성 점수가 포함되어 있습니다.
senti_dict = pd.read_csv('/content/SentiWord_dict.csv', encoding='cp949')

# 단어의 감성 점수를 반환하는 함수입니다.
def get_sentiment(word):
    matched_words = senti_dict[senti_dict['word'] == word]  # 사전에서 단어를 찾습니다.
    if matched_words.empty:  # 사전에 단어가 없으면 None을 반환합니다.
        return None
    return matched_words.iloc[0]['polarity']  # 단어의 감성 점수(polarity)를 반환합니다.

# 감성 분석을 실행하고 결과를 저장합니다.
results = []  # 결과를 저장할 리스트
for idx, row in data.iterrows():  # 데이터프레임의 모든 행에 대해 반복
    for word in row['preprocessing'].split():  # 전처리된 텍스트를 공백 기준으로 분리하여 각 단어에 대해 반복
        polarity = get_sentiment(word)  # 단어의 감성 점수를 얻습니다.
        if polarity is not None:  # 감성 사전에 단어가 있으면 결과 리스트에 추가
            results.append({
                'document_id': idx,  # 문서 ID
                'preprocessing': row['preprocessing'],  # 전처리된 텍스트
                'word': word,  # 단어
                'polarity': polarity  # 단어의 감성 점수
            })

# 결과를 DataFrame으로 변환하고 CSV 파일로 저장합니다.
results_df = pd.DataFrame(results)
results_df.to_csv('sentiment_analysis_results.csv', index=False, encoding='cp949')

In [17]:
# 전체 단어 수, 긍정적인 단어 수, 부정적인 단어 수를 계산합니다.
total_words = len(results_df)  # 전체 결과의 수를 세어서 저장합니다.
pos_count = len(results_df[results_df['polarity'] > 0])  # 긍정적인 결과만 필터링하고 그 수를 세어서 저장합니다.
neg_count = len(results_df[results_df['polarity'] < 0])  # 부정적인 결과만 필터링하고 그 수를 세어서 저장합니다.
neutral_count = len(results_df[results_df['polarity'] == 0])  # 중립적인 결과만 필터링하고 그 수를 세어서 저장합니다.

print("긍정단어개수:",pos_count)
print("부정단어개수:",neg_count)
print("중립단어개수:",neutral_count)
# 긍정적인 단어와 부정적인 단어의 비율을 계산합니다.
pos_proportion = (pos_count / total_words) * 100 if total_words else 0  # 긍정적인 단어의 비율을 계산합니다.
neg_proportion = (neg_count / total_words) * 100 if total_words else 0  # 부정적인 단어의 비율을 계산합니다.
neutral_proportion = (neutral_count / total_words) * 100 if total_words else 0  # 중립적인 단어의 비율을 계산합니다.

print(pos_proportion)
print(neg_proportion)
print(neutral_proportion)


긍정단어개수: 1309
부정단어개수: 470
중립단어개수: 22
72.68184342032204
26.096612992781786
1.2215435868961688
